# Chicago taxi tips Regression Pipeline local

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def create_sets(data,label_column):
    ### get label
    label=data[label_column]
    ### get features
    data=data.drop([label_column],axis=1)
    ### split
    X_train, X_test, y_train, y_test = train_test_split(..., ..., test_size=0.2, random_state=100)
    ### group per train, test
    dtrain = xgboost.DMatrix(..., label=...)
    dtest = xgboost.DMatrix(..., label=...)
    return dtrain, ... 
    

#### Create callback to use xgboost with tensorboard

In [ ]:
!pip install tensorboardX

In [ ]:
from tensorboardX import SummaryWriter

In [ ]:
def TensorBoardCallback(training_log_path):
    writer = SummaryWriter(training_log_path)

    def callback(env):
        for k, v in env.evaluation_result_list:
            print(k,v)
            writer.add_scalar(k, v, env.iteration)

    return callback

#### Create the main train function

In [ ]:
!pip install xgboost==1.1.0

In [ ]:
import xgboost

In [ ]:
def xgboost_train(data,label,num_iterations,training_log_path,booster_params):
    
    ### create train,test objects 
    dtrain, dtest = create_sets(...,...)
    
    # Booster parameters with default values
    booster_params = booster_params or {}
    booster_params.setdefault('objective', 'reg:squarederror')
    booster_params.setdefault('booster', 'gbtree')
    booster_params.setdefault('learning_rate', ...)
    booster_params.setdefault('min_split_loss', 0)
    booster_params.setdefault('max_depth', 6)
    
    num_iterations = num_iterations or 20
    
    ### create blank model
    model = xgboost.train(
        params=...,
        dtrain=...,
        num_boost_round=...,
        evals=[(dtrain, 'train'), (dtest, 'test')],
       callbacks=[TensorBoardCallback(training_log_path)]
    )
    return ...

### Test the training

##### Create minio client to get data

In [ ]:
!pip install pyarrow

In [ ]:
from minio import Minio
import urllib3
from io import BytesIO
import pandas as pd
import pyarrow
import datetime
import os

In [ ]:
## Create a client with the access key and the secret key given
## access_key=os.getenv("MINIO-ACCESS-KEY"),
## secret_key=os.getenv("MINIO-SECRET-KEY"),
client = Minio(

    ...

)

In [ ]:
bucket_name = 'xxx-xxx'
object_name = 'datasets/chicago/trips.parquet'

In [ ]:
# Get data from minio using get_object, decode it using BytesIO and read the parquet result with pandas
try:
    response = client.get_object(..., ..)
    # Read data from response.
    parquet_object=BytesIO(...)
    data = pd.read_...
finally:
    response.close()
    response.release_conn()

In [ ]:
data.head()

In [ ]:
data = data.drop(["trip_start_timestamp"],axis=1)

In [ ]:
xgboost_train(
    data=...,
    label=..., # what column we want to predict ? 
    num_iterations=20,    
    training_log_path=f"training/fit/taxi-trips/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}",
    booster_params={}
)

### Now connect to your tensorboard and check the loss

1. Go to the Kubeflow interface and click on 'tensorboard'

![menu_tenso](./images/menu_tenso.png)

2. click on 'new' 

![new](./images/new.png)

3. link it to your lab, where you persist your training logs

![board](./images/board.png)

4. click `connect` and you should be able to see your train metrics

![tenso](./images/tenso.png)